In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime
import hashlib
import os


In [5]:
DB_DIR = r"C:/Users/HP/Documents/repos/news_ingestion_data_pipeline/data"
DB_FILE = "articles.db"
DB_PATH = os.path.join(DB_DIR, DB_FILE)

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/115.0.0.0 Safari/537.36'
}

In [35]:
def get_latest_news_time():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.execute("SELECT MAX(News_published_time) FROM articles")
        result = cursor.fetchone()
        if result[0]:
            incremental = True
            latest_timestamp = result[0]
        else:
            incremental = False
            latest_timestamp = '2025-07-25T00:00:00'
        return (latest_timestamp, incremental)
    
get_latest_news_time()

('2025-08-01T00:00:00', True)

In [8]:
def parse_date(date_str, source= 'skift'):
    """
    Converts date string like 'July 28, 2025' to datetime object.
    Return None if parse fails.
    """
    try:
        if source == "Phocusewire":
            return datetime.strptime(date_str.strip(), "%B %d, %Y")
        else:
            return datetime.fromisoformat(date_str)
    except Exception:
        return None
    
def generate_article_id(url):
    return hashlib.md5(url.encode("utf-8")).hexdigest()



def datetime_to_iso_with_time(dt, time_str="00:00:00"):
    """
    Convert a datetime object to ISO8601 string with a fixed time part.

    Args:
        dt (datetime): A datetime object (date part used).
        time_str (str): Time string 'HH:MM:SS' to append (default '10:00:00').

    Returns:
        str: Formatted ISO8601 string in 'YYYY-MM-DDTHH:MM:SS' format
    """
    date_part = dt.strftime("%Y-%m-%d")
    return f"{date_part}T{time_str}"

In [36]:
def phocuswire_web_scraping(last_ingested_date):
    """
    Scrapes articles page by page, stops when articles are older than last_ingested_date.

    Args:
        last_ingested_date (datetime or None): datetime to stop ingestion at.
        max_pages (int): max number of pages to crawl.

    Returns:
        List of BeautifulSoup article elements (or could be dictionaries of data).
    """
    base_url = "https://www.phocuswire.com"
    page = 1
    collected_articles = []
    last_ingested_date = parse_date(last_ingested_date)
    while True:
        base_url = "https://www.phocuswire.com"
        url = f"{base_url}/Latest-News?pg={page}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Stopping due to bad status: {response.status_code} at page {page}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.select("div.article-list  div.item")

        if not articles:
            print(f"No articles found on page {page}, stopping.")
            break

        stop_paging = False

        for article in articles:
            # Extract news date (inside div.author)
            title_tag = article.select_one("a.title")
            headline = title_tag.get_text(strip=True) if title_tag else None
            news_url = f"{base_url}/{title_tag['href']}" if title_tag and 'href' in title_tag.attrs else None
            article_id = generate_article_id(news_url) if news_url else None

            author_span = article.select_one("div.author > span.name")
            author_name = author_span.get_text(strip=True).replace("By ", "") if author_span else None
            
            # Extract news time from div.author text after the pipe symbol
            author_div = article.select_one("div.author")
            news_time = None
            if author_div:
                # The text looks like 'By Abby Crotty | July 28, 2025'
                # We can split by '|' and strip whitespace to get news time
                parts = author_div.text.split('|')
                if len(parts) == 2:
                    news_time = parts[1].strip()
                    news_time = parse_date(news_time, "Phocusewire")

            if news_time:
                # If last_ingested_date is set and this article is older or equal, stop ingestion
                print("News time : ",news_time)
                print("last -ngested time: ", last_ingested_date)
                print("-"*40)
                if last_ingested_date and news_time < last_ingested_date:
                    stop_paging = True
                    print(f"Encountered article dated {news_time} < last ingested {last_ingested_date}, stopping.")
                    break
            else:
                # If no date found, you can decide to skip or include
                print("Article without date found, skipping date check.")

            collected_articles.append({"Article_id":article_id,"News_title":headline,"News_link":news_url,"Author_name":author_name,"News_published_time":datetime_to_iso_with_time(news_time), "Source_name": "Phocuswire"})

        if stop_paging:
            break

        page += 1
        print(page)

    print("No. of new articles ingested : ", len(collected_articles))
    return collected_articles

In [37]:
latest_timestamp = get_latest_news_time()
print(latest_timestamp)
extracted_articles = phocuswire_web_scraping(latest_timestamp[0])

('2025-08-01T00:00:00', True)
News time :  2025-08-01 00:00:00
last -ngested time:  2025-08-01 00:00:00
----------------------------------------
News time :  2025-08-01 00:00:00
last -ngested time:  2025-08-01 00:00:00
----------------------------------------
News time :  2025-08-01 00:00:00
last -ngested time:  2025-08-01 00:00:00
----------------------------------------
News time :  2025-07-31 00:00:00
last -ngested time:  2025-08-01 00:00:00
----------------------------------------
Encountered article dated 2025-07-31 00:00:00 < last ingested 2025-08-01 00:00:00, stopping.
No. of new articles ingested :  3


In [38]:
extracted_articles

[{'Article_id': '57df0278655170f8de688fcd84bbd72b',
  'News_title': 'The evolution of CRM in hotels',
  'News_link': 'https://www.phocuswire.com//penta-hotels-smart-host-phocuswright-europe-2025',
  'Author_name': 'Linda Fox',
  'News_published_time': '2025-08-01T00:00:00',
  'Source_name': 'Phocuswire'},
 {'Article_id': '6c2fe2b95e5b4febb64ba2105cf07582',
  'News_title': 'Weighing in on the next generation of travel products and services around the world',
  'News_link': 'https://www.phocuswire.com//phocuswright-europe-2025-bridge-series-next-gen-travel-products-services',
  'Author_name': 'Abby Crotty',
  'News_published_time': '2025-08-01T00:00:00',
  'Source_name': 'Phocuswire'},
 {'Article_id': '7ac8bd31b72b6538309c54d643853517',
  'News_title': "PhocusWire's weekly travel tech news briefs: Casago, GuideGeek, Booking.com and more...",
  'News_link': 'https://www.phocuswire.com//travel-tech-news-briefs-aug-1-2025',
  'Author_name': 'PhocusWire',
  'News_published_time': '2025-08-01

In [27]:
def upsert_articles(filtered_articles):

    with sqlite3.connect(DB_PATH) as conn:
        for article in filtered_articles:
            sql = """
            INSERT INTO articles (Article_id,  News_link, News_title, Author_name, News_published_time, Source_name, Processed_at)
            VALUES (?, ?,  ?, ?, ?, ?, CURRENT_TIMESTAMP)
            ON CONFLICT(Article_id) DO UPDATE SET
                News_link = excluded.News_link,
                News_title = excluded.News_title,
                Author_name = excluded.Author_name,
                News_published_time = excluded.News_published_time,
                Source_name = excluded.Source_name,
                Processed_at = CURRENT_TIMESTAMP
            """
            params = (
                article.get('Article_id'),
                article.get('News_link'),
                article.get('News_title'),
                article.get('Author_name'),
                article.get('News_published_time'),
                article.get('Source_name'),
            )
            print(params)
            cursor = conn.cursor()
            cursor.execute(sql, params)

In [39]:
upsert_articles(extracted_articles)

('57df0278655170f8de688fcd84bbd72b', 'https://www.phocuswire.com//penta-hotels-smart-host-phocuswright-europe-2025', 'The evolution of CRM in hotels', 'Linda Fox', '2025-08-01T00:00:00', 'Phocuswire')
('6c2fe2b95e5b4febb64ba2105cf07582', 'https://www.phocuswire.com//phocuswright-europe-2025-bridge-series-next-gen-travel-products-services', 'Weighing in on the next generation of travel products and services around the world', 'Abby Crotty', '2025-08-01T00:00:00', 'Phocuswire')
('7ac8bd31b72b6538309c54d643853517', 'https://www.phocuswire.com//travel-tech-news-briefs-aug-1-2025', "PhocusWire's weekly travel tech news briefs: Casago, GuideGeek, Booking.com and more...", 'PhocusWire', '2025-08-01T00:00:00', 'Phocuswire')


In [32]:
def query_top5_articles():
    # Connect to the SQLite database
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()

        # Execute a query to select all articles
        cursor.execute("""SELECT Article_id, News_link, News_title,Author_name, News_published_time, Source_name, Processed_at FROM articles
                        ORDER BY News_published_time DESC LIMIT 5""")

        # Fetch all rows returned by the query
        rows = cursor.fetchall()
        # Process and display results
        for row in rows:
            print(row)

In [41]:
with sqlite3.connect(DB_PATH) as conn:
    cursor = conn.cursor()

    # Execute a query to select all articles
    cursor.execute("""SELECT Count(*) FROM articles
                    """)

    # Fetch all rows returned by the query
    rows = cursor.fetchall()
    # Process and display results
    for row in rows:
        print(row)

(17,)


In [33]:
query_top5_articles()


('57df0278655170f8de688fcd84bbd72b', 'https://www.phocuswire.com//penta-hotels-smart-host-phocuswright-europe-2025', 'The evolution of CRM in hotels', 'Linda Fox', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:21:22')
('6c2fe2b95e5b4febb64ba2105cf07582', 'https://www.phocuswire.com//phocuswright-europe-2025-bridge-series-next-gen-travel-products-services', 'Weighing in on the next generation of travel products and services around the world', 'Abby Crotty', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:21:22')
('7ac8bd31b72b6538309c54d643853517', 'https://www.phocuswire.com//travel-tech-news-briefs-aug-1-2025', "PhocusWire's weekly travel tech news briefs: Casago, GuideGeek, Booking.com and more...", 'PhocusWire', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:21:22')
('8f1bc8e9aa9412e967c9b535a7020a6b', 'https://www.phocuswire.com//amadeus-q2-2025', 'Amadeus looks to future with ‘steady growth’ in first half of 2025', 'Linda Fox', '2025-07-31T00:00:00', 'Phocuswire', '

In [40]:
query_top5_articles()


('57df0278655170f8de688fcd84bbd72b', 'https://www.phocuswire.com//penta-hotels-smart-host-phocuswright-europe-2025', 'The evolution of CRM in hotels', 'Linda Fox', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:25:22')
('6c2fe2b95e5b4febb64ba2105cf07582', 'https://www.phocuswire.com//phocuswright-europe-2025-bridge-series-next-gen-travel-products-services', 'Weighing in on the next generation of travel products and services around the world', 'Abby Crotty', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:25:22')
('7ac8bd31b72b6538309c54d643853517', 'https://www.phocuswire.com//travel-tech-news-briefs-aug-1-2025', "PhocusWire's weekly travel tech news briefs: Casago, GuideGeek, Booking.com and more...", 'PhocusWire', '2025-08-01T00:00:00', 'Phocuswire', '2025-08-02 12:25:22')
('8f1bc8e9aa9412e967c9b535a7020a6b', 'https://www.phocuswire.com//amadeus-q2-2025', 'Amadeus looks to future with ‘steady growth’ in first half of 2025', 'Linda Fox', '2025-07-31T00:00:00', 'Phocuswire', '

In [ ]:
1) Query target data source, sqlite, and get max time stamp from the data base. -Done
2) If timestamp is null return latest_timestamp = '2025-07-01T00:00:00' and incremental = False (meaning do full load from this timestamp)
 else return latest_timestamp from database and incremental = True. -Done
3) Now since website is paginated now query the website from latest page and check for the last news on each page is that smaller then latest_timestamp 
if yes then stop else go to next page and repeat process.
4) once you get all the extracted information, just merge the the extracted information in the database so to avoid any duplicates on last page of news.

This will be repeated for the other source too.

In [ ]:
def phocuswire_web_scraping(last_ingested_date, min_delay=1, max_delay=3):
    """
    Scrapes articles page by page, stops when articles are older than last_ingested_date.

    Args:
        last_ingested_date (datetime or None): datetime to stop ingestion at.
        max_pages (int): max number of pages to crawl.

    Returns:
        List of BeautifulSoup article elements (or could be dictionaries of data).
    """
    base_url = "https://www.phocuswire.com"
    page = 1
    collected_articles = []
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/115.0.0.0 Safari/537.36'
    }
    while True:
        base_url = "https://www.phocuswire.com"
        url = f"{base_url}/Latest-News?pg={page}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Stopping due to bad status: {response.status_code} at page {page}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.select("div.article-list  div.item")

        if not articles:
            print(f"No articles found on page {page}, stopping.")
            break

        stop_paging = False

        for article in articles:
            # Extract news date (inside div.author)
            title_tag = article.select_one("a.title")
            headline = title_tag.get_text(strip=True) if title_tag else None
            news_url = f"{base_url}/{title_tag['href']}" if title_tag and 'href' in title_tag.attrs else None
            article_id = generate_article_id(news_url) if news_url else None

            author_span = article.select_one("div.author > span.name")
            author_name = author_span.get_text(strip=True).replace("By ", "") if author_span else None
            
            # Extract news time from div.author text after the pipe symbol
            author_div = article.select_one("div.author")
            news_time = None
            if author_div:
                # The text looks like 'By Abby Crotty | July 28, 2025'
                # We can split by '|' and strip whitespace to get news time
                parts = author_div.text.split('|')
                if len(parts) == 2:
                    news_time = parts[1].strip()
                    news_time = parse_date(news_time, "Phocusewire")

            if news_time:
                # If last_ingested_date is set and this article is older or equal, stop ingestion
                logging.info(f"News time : {news_time}")
                logging.info(f"last -ngested time: {last_ingested_date}")

                if last_ingested_date and news_time < last_ingested_date:
                    stop_paging = True
                    print(f"Encountered article dated {news_time} < last ingested {last_ingested_date}, stopping.")
                    break
            else:
                # If no date found, you can decide to skip or include
                print("Article without date found, skipping date check.")

            collected_articles.append({"Article_id":article_id,"News_title":headline,"News_link":news_url,"Author_name":author_name,"News_published_time":datetime_to_iso_with_time(news_time), "Source_name": "Phocuswire"})

        if stop_paging:
            break

        page += 1
        print(page)

    print("No. of new articles ingested : ", len(collected_articles))
    return collected_articles